# 4.6 無情報事前分布

## インポート・設定

In [1]:
import dataclasses
import warnings
from numbers import Number

import arviz as az
import numpy as np
import pandas as pd
import pymc as pm
import seaborn as sns
from IPython.display import Math
from scipy import stats

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
sns.set_theme()
warnings.simplefilter("ignore")
RANDM_STATE = np.random.default_rng(seed=0)